In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # for kde
import scipy # for sparse arrays

# Ripser allows us to use a sparse filtration to compute PH. 
import ripser 
from gudhi.wasserstein import wasserstein_distance

import tqdm # for showing progress bars

In [ ]:
def two_wasserstein(dgmx, dgmy, gamma):
    
    """
    Calculate the 2-wasserstein distance between persistence diagrams dgmx and dgmy, 
    and scale by a factor of gamma. 
    
    Gudhi's wasserstein doesn't handle cases where one of the diagrams has
    only one persistence point, so it has to be done manually.
    """

    l1 = dgmx.shape
    l2 = dgmy.shape

    # First diagram has more than one persistence point
    if l1 != (2,):
        
        # So does the second diagram, normal wasserstein distance works
        if l2 != (2,):
            return -gamma * wasserstein_distance(dgmx, dgmy, False, 2.0, 2.0)
        
        # Otherwise dgmy has only one persistence point, have to compute manually
        # Unsure if this step is right ??? 
        else:
            infinite_cost = (dgmx[-1, 0] - dgmy[0]) ** 2
            diagonal_cost = (dgmx[:-1, 1] - dgmx[:-1, 0]) * 0.5
            return -gamma * np.sqrt(infinite_cost + np.sum(diagonal_cost))

    # First diagram only has one persistence point
    else:
        # Second diagram has more than one persistance point, we compute manually.
        if l2 != (2,):
            infinite_cost = (dgmx[0] - dgmy[-1,0]) ** 2
            diagonal_cost = (dgmy[:-1, 1] - dgmy[:-1, 0]) * 2 * 0.5
            return -gamma * np.sqrt(infinite_cost + np.sum(diagonal_cost))
        
        # Both diagrams only have one persistence point, the distance is the di
        else:
            return -gamma *np.abs(dgmx[0] - dgmy[0])
        

In [ ]:
class GraphPersistenceGenerator:

    """

    This class works in the following way:
        
        We rely on the creation of a custom filtration to preform persistent homology computations.
        This filtration takes the form of a matrix which inherits the structure of G, i.e.  
    
        The diagonal of this matrix will be given by node values. If G has an edge between nodes 
        (i,j), then we set filtration[i,j] to be the maximum node value between i and j. If there is not
        an edge between nodes (i,j), then we treat the "distance" as infinite.

        Persistent homology calculations are 

    """
    def __init__(self, G):

        """
        Initialize the adjacency matrix
        G : networkx graph
        
        """
        
        adj = nx.adjacency_matrix(G)
        h, w = adj.shape
        
        # Get indices of edges, determined by the structure on G
        # edge_idx, edge_idy = adj.nonzero(). Of course, the adjacency matrix is symmertric; 
        # for efficiency, we choose to remember only the lower diagonal indcies to update the filtration. 

        inds = np.vstack([*adj.nonzero()]).T

        full_edge_idx, full_edge_idy = inds[:, 0], inds[:, 1]
        
        lower_diag_inds = inds[np.argmax(inds, axis=1) > 0, :]
        edge_idx, edge_idy = lower_diag_inds[:, 0], lower_diag_inds[:, 1]
        
        # Indices of nodes, which are on the diagonal of the filtration matrix 
        node_idx, node_idy = np.arange(0,h), np.arange(0,h)

        # Combined indices of edges and nodes. Note that we need to account for the symmetry of the adjacency
        # matrix here, which is why we are using the full_edge_idx and full_edge_idy here.
        idx_x, idx_y = np.concatenate([full_edge_idx, node_idx]), np.concatenate([full_edge_idy, node_idy])
        self.filtration_inds = list(zip(np.concatenate([edge_idx, node_idx]), np.concatenate([edge_idy, node_idy])))
        
        self.filtration = scipy.sparse.csr_matrix((np.ones(len(idx_x)), (idx_x, idx_y)), shape=(h,w))

    def update(self, new_node_values):
        
        """
        Update edge weights/node values with new_node_values
        For example, suppose the current state of self.filtration is 
        
        [[0.1, 0.4, 0.0], 
         [0.4, 0.4, 0.4],
         [0.0, 0.4, 0.1]]
        
        and the indicies we replace are specified as [[0,0], [0,1], [1,1], [2,1], [2,2]].
        
        If new_node_values = [0.05, 0.1, 0.25], 
        calling self.update() will change self.filtration to
        
        [[0.05, 0.1, 0.0], 
         [0.1, 0.1, 0.25],
         [0.0, 0.25, 0.25]]


        Note that for a given edge in the filtration, which is specified by indices [i,j], the 
        edge will be "created" at max(node_value(i), node_value(j)) when we want to 
        calculate the persistence diagram of the graph. 
        
        """

        for (i,j) in self.filtration_inds:
            new_edge_weight = np.maximum(new_node_values[i], new_node_values[j])
            self.filtration[i,j] = new_edge_weight
            self.filtration[j,i] = new_edge_weight
        
        
    def compute_PH(self):
        
        """
        Compute the 0-dimensional persistent homology for the current filtration.
        
        Ripser returns a list of persistence diagram points, which we need to convert to a 2-D numpy array to
        use in gudhi's wasserstein_distance function.
        """
        
        return np.squeeze(np.array(ripser.ripser(self.filtration, maxdim=0, distance_matrix=True)['dgms']))


    def __call__(self, new_node_values):
        self.update(new_node_values)
        return self.compute_PH()

    
        

In [ ]:
class EigenvectorRandomWalkMetropolis:

    """
    
    Graph laplacian eigenvector Random Walk Metropolis Class
    
    parameters: 

        G : networkx graph
        
        target : the target persistence diagram (n x 2 numpy array)
        
        start : the starting point for mcmc

        step : Proposed moves follow a normal distribution with mean = 0 and std = step.
        
        nev : number of graph laplacian eigenvectors to take - this is the dimension 
              of the search space
              
        MIN/MAX : bounds of the search space 

        
        gamma : scaling factor for 2-wasserstein distance
        
    """

    def __init__(self, G, target, start, step, nev, MIN=-np.inf, MAX=np.inf, gamma=1.0):

        laplacian = nx.normalized_laplacian_matrix(G).toarray()
        _, eigenvectors = np.linalg.eigh(laplacian)

        self.phi = eigenvectors[:, 0:nev]
        self.gpg = GraphPersistenceGenerator(G)
        self.target = target
        self.current_location = start
        self.step = step
        self.gamma = gamma
        self.nev = nev
        self.current_p = self.density_func(start)
        self.num_accepted = 0
        self.lower_bound = MIN
        self.upper_bound = MAX

    def density_func(self, x):

        """ 
        
        x is assumed to be a vector of coefficients in this case, but
        different varieties of density functions can replace this
        
        """

        output_pd = self.gpg(self.phi @ x)
        return two_wasserstein(output_pd, self.target, self.gamma)
    
    def sample(self):

        """Preform one iteration of MCMC"""
        
        proposed_move = np.random.normal(loc=0.0, scale=step, size=self.nev) + self.current_location
        if np.all(proposed_move >= self.lower_bound) & np.all(proposed_move <= self.upper_bound):
            proposed_move_density = self.density_func(proposed_move)
            
            if np.random.uniform() < np.exp(proposed_move_density - self.current_p):
                
                # the move is accepted, update the current location and density, and
                # increment the numeber of accepted moves by one.
                self.current_p = proposed_move_density
                self.current_location = proposed_move
                self.num_accepted += 1


In [ ]:
class MCMC:

    """
    
    Runs MCMC over multiple iterations, and keeps track of paramter locations and densities. 
    Also inlcudes functionality for plotting trace plots 

        - model should be one of the RWM models in this file.
        - dim is the dimension of the search space
        - gamma is the 2-wasserstein penalty term used in the density function, defaults to 1.0
        - iterations - number of mcmc iterations to run
        - burnin - burnin period
        - seed - random seed for reproducibility.
        
    """

    def __init__(self, model, dim, gamma=1.0, iterations=100000, burnin=10000, seed=42):

        self.model = model
        
        self.iterations = iterations
        self.burnin = burnin
        
        # Set random seed for reproducibility
        self.seed = seed

        self.sim = np.zeros((dim, iterations-burnin))
        self.losses = np.zeros(iterations-burnin)

        # If we are using a penalty term when calculating the 2-wasserstein distance, include it
        # here to scale densities to calculate various performance metrics.
        self.gamma = gamma
        
    def run(self):

        """ 
        Run MCMC for the specified number of iterations
        """
        
        np.random.seed(self.seed)
        for i in tqdm.tqdm(range(self.iterations)):

            self.model.sample()

            if i - self.burnin >= 0:
                self.sim[:, i - burnin] = model.current_location
                self.losses[i - burnin] = - model.current_p / self.gamma

        print(f"Percentage of moves accepted : {model.num_accepted / self.iterations * 100.0} %")
        print(f"Best Loss: {np.min(self.losses)}")


    
    def trace_plots(self, inds=None):
        
        """
        Create trace plots
        
        inds : the parameters you want to see trace plots for - e.g, if you have 
               8 parameters, and you pass [0,1,2,3] to trace_plots, you will see 
               trace plots for the first 4 parameters. 

               Passing None will result in all trace plots being shown
        
        """    
        
        if inds:
            fig, axes = plt.subplots(len(inds), 1, layout='constrained')
        else:
            inds = range(self.sim.shape[0])
            fig, axes = plt.subplots(self.sim.shape[0], 1, layout='constrained')
        
        for (i,indx) in enumerate(inds):
            sns.lineplot(data=self.sim[i, :], ax=axes[i])        
      
        plt.show()

    def density_plots(self, inds=None):
        """Create density plots, inds functions the same as in trace_plots"""
        
        if inds:
            fig, axes = plt.subplots(len(inds), 1, layout='constrained')
        else:
            inds = range(self.sim.shape[0])
            fig, axes = plt.subplots(self.sim.shape[0], 1, layout='constrained')
        
        for (i,indx) in enumerate(inds):
            sns.kdeplot(data=self.sim[i, :], ax=axes[i]) 

        plt.show()
        
    def best_approximation(self):
        """Return the node values of the best apporximation found in the run"""
        min_loss = np.argmin(self.losses)
        best_coeff = self.sim[:, min_loss]
        best_node_values = self.model.phi @ best_coeff

        return best_node_values

In [ ]:
nev = 6
iterations = 1000
burnin = 100
gamma = 40.0
seed = 42

num_nodes = 14
node_prob = 0.2

G = nx.erdos_renyi_graph(num_nodes, node_prob, seed=seed)
laplacian = nx.normalized_laplacian_matrix(G).toarray()
_, eigenvectors = np.linalg.eigh(laplacian)

phi = eigenvectors[:, 0:nev]

nx.draw(G)

In [ ]:
# G, target, start, step, nev, MIN=-np.inf, MAX=np.inf, gamma=1.0

np.random.seed(45)
graph_persistence_generator = GraphPersistenceGenerator(G)
target_node_values = np.random.rand(num_nodes)

target = graph_persistence_generator(target_node_values)
start = np.random.normal(size=nev)
step = 0.1
MIN = -np.inf
MAX = np.inf

model = EigenvectorRandomWalkMetropolis(G, target, start, step, nev, MIN, MAX, gamma)


In [ ]:
simulation = MCMC(model, nev, gamma, iterations, burnin, seed)
simulation.run()

In [ ]:
simulation.density_plots()

In [ ]:
best_coeff = simulation.best_approximation()